In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
from statistics import mean

dir_ = './data/'

In [2]:
file_name = 'normalized_minmax_filter_track_5_user_100.csv'
pred_m = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test_m = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

file_name = 'normalized_log_filter_track_5_user_100.csv'
pred_log = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test_log = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
pred_l = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test_l = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

num_user = len(pred_m['uid'].unique())
num_user

953

In [3]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [4]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [5]:
# Personal Recommendation
n=50
top_n_lists_m = []
top_n_lists_l = []
top_n_lists_log = []

for i in tqdm(range(num_user)):
    pred = pred_m[pred_m['uid'] == i]
    pred = pred.sort_values(by=['rating'],  ascending=False)
    pred = pred[:n]
    top_n_list_m = []
    for _, row in pred.iterrows():
        top_n_list_m.append(row[1])
    top_n_lists_m.append(top_n_list_m)
    
    pred = pred_l[pred_l['uid'] == i]
    pred = pred.sort_values(by=['rating'],  ascending=False)
    pred = pred[:n]
    top_n_list_l = []
    for _, row in pred.iterrows():
        top_n_list_l.append(row[1])
    top_n_lists_l.append(top_n_list_l)
    
    pred = pred_log[pred_log['uid'] == i]
    pred = pred.sort_values(by=['rating'],  ascending=False)
    pred = pred[:n]
    top_n_list_log = []
    for _, row in pred.iterrows():
        top_n_list_log.append(row[1])
    top_n_lists_log.append(top_n_list_log)

In [6]:
precision = []
recall = []
nDCG = []
NOV = []
HitNOV = []

test = test_m
covs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
hitCovs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists_m[i]
    
    hit = 0
    nov = 0
    hitnov = 0
    truth_rating = []
    for j in top_n_list:
        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        nov += (1-pop_dict[j])
        covs[int(j)] = True
        if len(t) > 0:
            hit += 1
            hitnov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hitCovs[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    NOV.append(nov/n)
    HitNOV.append(hitnov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('NOV', mean(NOV))
print('HitNOV', mean(HitNOV))
print('COV', list(covs).count(True)/len(all_user['tid'].unique()))
print('HitCOV', list(hitCovs).count(True)/len(all_user['tid'].unique()))


presicion:  0.11945435466946484
recall:  0.011246228196379228
nDCG:  0.051742824298940955
NOV 0.6576014992143878
HitNOV 0.07228140218826283
COV 0.0035159646372654174
HitCOV 0.0021958912716495203


In [7]:
precision = []
recall = []
nDCG = []
NOV = []
HitNOV = []

test = test_l
covs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
hitCovs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists_l[i]
    
    hit = 0
    nov = 0
    hitnov = 0
    truth_rating = []
    for j in top_n_list:
        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        nov += (1-pop_dict[j])
        covs[int(j)] = True
        if len(t) > 0:
            hit += 1
            hitnov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hitCovs[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    NOV.append(nov/n)
    HitNOV.append(hitnov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('NOV', mean(NOV))
print('HitNOV', mean(HitNOV))
print('COV', list(covs).count(True)/len(all_user['tid'].unique()))
print('HitCOV', list(hitCovs).count(True)/len(all_user['tid'].unique()))


presicion:  0.12329485834207765
recall:  0.010897588589790062
nDCG:  0.06855756490576671
NOV 0.6621893859232841
HitNOV 0.07534532249735468
COV 0.010789061161283772
HitCOV 0.003154213763034138


In [8]:
precision = []
recall = []
nDCG = []
NOV = []
HitNOV = []

test = test_log
covs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
hitCovs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists_log[i]
    
    hit = 0
    nov = 0
    hitnov = 0
    truth_rating = []
    for j in top_n_list:
        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        nov += (1-pop_dict[j])
        covs[int(j)] = True
        if len(t) > 0:
            hit += 1
            hitnov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hitCovs[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    NOV.append(nov/n)
    HitNOV.append(hitnov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('NOV', mean(NOV))
print('HitNOV', mean(HitNOV))
print('COV', list(covs).count(True)/len(all_user['tid'].unique()))
print('HitCOV', list(hitCovs).count(True)/len(all_user['tid'].unique()))


presicion:  0.13173137460650577
recall:  0.011694271020231593
nDCG:  0.08633387587885927
NOV 0.6494413510546581
HitNOV 0.07937060742626421
COV 0.0045694847271319505
HitCOV 0.0023291679095242025


In [9]:
n=50
file_name = 'normalized_popularity_filter_track_5_user_100.csv'
pop_count = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))

In [10]:
pop_c = pop_count.sort_values(by=['count'], ascending= False)[:n]

In [11]:
all_user['c'] = 1
pop_u = all_user.groupby(["tid"]).sum().reset_index()
pop_u = pop_u[['tid', 'count', 'c']]

In [12]:
pop_u = pop_u.sort_values(by=['c'], ascending= False)[:n]

In [13]:
pop_c_list = pop_c['tid'].values
pop_c_list

array([   70,  1102,    83, 13496,   952,  2519,  4008,  5521,  3900,
        2241,   390,   210,  4374,  2917,  3852,  1628,  5716,  7031,
        1464,  1987,  6352, 15008,  1036,  3006,  3849,  1559,  1056,
        3735,  9638,   618,  4323,  1109,  1930,  3835,  4302,  4295,
        2947,  4322,  5917,  1806,  6509, 15058,  5389,  1335, 16244,
        8910,  5236, 11710,   810,  5268])

In [14]:
pop_u_list = pop_u['tid'].values
pop_u_list

array([   70,  5521,   390,  2519,    83, 13496,   210,  5716,  1036,
        1464,  3006,  2917,   952,  3900,  1987,  4302,  1335,  1102,
        5389,  3735,  6509,  6831,  5494,  4295, 11710,  6352,  5236,
        1628,   618,  1559, 15008,  8865,  3370,  4013,   713,  4374,
       16244,  5268,  4664,  3852,   810,  1056,  5590,  4322,  3822,
        7031,  2261,  2499,  6891,  8910])

In [15]:

precision = []
recall = []
nDCG = []
NOV = []
HitNOV = []

test = test_log
covs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
hitCovs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    
    hit = 0
    nov = 0
    hitnov = 0
    truth_rating = []
    for j in pop_c_list:
        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        nov += (1-pop_dict[j])
        covs[int(j)] = True
        if len(t) > 0:
            hit += 1
            hitnov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hitCovs[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    NOV.append(nov/n)
    HitNOV.append(hitnov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('NOV', mean(NOV))
print('HitNOV', mean(HitNOV))
print('COV', list(covs).count(True)/len(all_user['tid'].unique()))
print('HitCOV', list(hitCovs).count(True)/len(all_user['tid'].unique()))


presicion:  0.09299055613850997
recall:  0.010154892617322915
nDCG:  0.058183338314470257
NOV 0.5490661070304304
HitNOV 0.04930184572053349
COV 0.0003173253282730521
HitCOV 0.0003173253282730521


In [16]:
precision = []
recall = []
nDCG = []
NOV = []
HitNOV = []

test = test_log
covs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
hitCovs = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    
    hit = 0
    nov = 0
    hitnov = 0
    truth_rating = []
    for j in pop_u_list:

        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        nov += (1-pop_dict[j])
        covs[int(j)] = True
        if len(t) > 0:
            hit += 1
            hitnov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hitCovs[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    NOV.append(nov/n)
    HitNOV.append(hitnov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('NOV', mean(NOV))
print('HitNOV', mean(HitNOV))
print('COV', list(covs).count(True)/len(all_user['tid'].unique()))
print('HitCOV', list(hitCovs).count(True)/len(all_user['tid'].unique()))


presicion:  0.09607555089192026
recall:  0.01039920363489501
nDCG:  0.0589089453429223
NOV 0.5304512067156346
HitNOV 0.049817233698410825
COV 0.0003173253282730521
HitCOV 0.0003173253282730521
